In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_probability as tfp

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(precision=4)
tf.__version__

'2.7.0'

In [3]:
# Use to create Train/Evaluation/Test sets

#pd_train_data = pd.read_csv('/content/drive/MyDrive/Expresso Churn/Train.csv')
#pd_train_data.columns

#-----Create a training and evaluation dataset-----

#eval_length = int(len(pd_train_data)*0.8)
#train_data = pd_train_data[:eval_length ]
#eval_data = pd_train_data[eval_length: ]

#train_data.to_csv("/content/drive/MyDrive/Expresso Churn/tfTrain.csv", index=False)
#eval_data.to_csv("/content/drive/MyDrive/Expresso Churn/tfEval.csv", index=False)

In [4]:
tf_train_data = pd.read_csv("/content/drive/MyDrive/Expresso Churn/tfTrain.csv")
tf_train_data.columns

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'ZONE1', 'ZONE2', 'MRG', 'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK',
       'CHURN'],
      dtype='object')

In [5]:
# Get unique tenures and regions to use later on for categorical encoding
unique_tenures = tf_train_data['TENURE'].unique().tolist()
unique_regions = tf_train_data['REGION'].unique().tolist()

In [6]:
CSV_COLUMNS = tf_train_data.columns
LABEL_COLUMN = 'CHURN'
DEFAULTS = [['new_user'],['not_given'],['not_given'],
            [0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],
            ['not_given'],[0.0],['not_given'],[0.0],[0]]

In [7]:
def feature_engg_features(features):
  # Add feature Engineering here

  return(features)
  

def feature_engg(features, label):
  #Add new features
  features = feature_engg_features(features)

  return(features, label) 

In [8]:
def create_input_fn(filename, mode, vnum_epochs = None, batch_size = 512):
  # filename can be a link/url to the file
  def _input_fn(v_test=False):
    file_list = tf.io.gfile.glob(filename)

    if mode == tf.estimator.ModeKeys.TRAIN:
      # If in training mode
      num_epochs = vnum_epochs 
    else:
      num_epochs = 1 # end-of-input after this

    #Create the dataset
    dataset = tf.data.experimental.make_csv_dataset(filename,                           
                                                    column_names=CSV_COLUMNS,
                                                    column_defaults=DEFAULTS,
                                                    batch_size=batch_size,
                                                    label_name=LABEL_COLUMN,
                                                    num_epochs=num_epochs,)
                                                    #ignore_errors=True,
    
      
    
    dataset = dataset.prefetch(buffer_size = batch_size)
    
    dataset = dataset.map(feature_engg)

    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = vnum_epochs # indefinitely
        dataset = dataset.shuffle(buffer_size = batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs)       
    
    #Begins - Uncomment for testing only -----------------------------------------------------<
    if v_test == True:
      print(next(dataset.__iter__()))
      
    #End - Uncomment for testing only -----------------------------------------------------<
    return dataset
  return _input_fn
                                                            


In [9]:
eval_file = "/content/drive/MyDrive/Expresso Churn/tfEval.csv"
fn_d = create_input_fn(filename = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 10)

fn_d(v_test=True)

(OrderedDict([('user_id', <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'cd594dd11db6cdc666e681223d9c1c4e38f3d7e3',
       b'cccdd540b8f829011e51949f8a3e2af01b7f46b4',
       b'cd2fc8c609e7ae1f7c57332ca0936164f52aa3f4',
       b'cd8dfaa386bbbfa3907ef5c161f11c370f069d8f',
       b'cd4e7a19bab42b47c4842d2f2ca33923ff6f3943',
       b'cd2c39ad3748fc40f8dd30b44df8f86a7e134498',
       b'ccff156275b4405767ddf80d9cbf2333b87e9535',
       b'cd1fbb1bc3569c4fd0d5c3930a2d9f4b38c627f1',
       b'cd7affc0c23d1ee00365faa5f15200b2852764d2',
       b'cce0ac547dca9676325dd7d4e1ce2f9935d2eb3a'], dtype=object)>), ('REGION', <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'SAINT-LOUIS', b'DAKAR', b'not_given', b'DIOURBEL', b'not_given',
       b'SAINT-LOUIS', b'not_given', b'KAOLACK', b'LOUGA', b'THIES'],
      dtype=object)>), ('TENURE', <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'I 18-21 month', b'K > 24 month', b'K > 24 month',
       b'H 15-18 month', b'I 18-21 month', b'K > 2

<RepeatDataset shapes: (OrderedDict([(user_id, (None,)), (REGION, (None,)), (TENURE, (None,)), (MONTANT, (None,)), (FREQUENCE_RECH, (None,)), (REVENUE, (None,)), (ARPU_SEGMENT, (None,)), (FREQUENCE, (None,)), (DATA_VOLUME, (None,)), (ON_NET, (None,)), (ORANGE, (None,)), (TIGO, (None,)), (ZONE1, (None,)), (ZONE2, (None,)), (MRG, (None,)), (REGULARITY, (None,)), (TOP_PACK, (None,)), (FREQ_TOP_PACK, (None,))]), (None,)), types: (OrderedDict([(user_id, tf.string), (REGION, tf.string), (TENURE, tf.string), (MONTANT, tf.float32), (FREQUENCE_RECH, tf.float32), (REVENUE, tf.float32), (ARPU_SEGMENT, tf.float32), (FREQUENCE, tf.float32), (DATA_VOLUME, tf.float32), (ON_NET, tf.float32), (ORANGE, tf.float32), (TIGO, tf.float32), (ZONE1, tf.float32), (ZONE2, tf.float32), (MRG, tf.string), (REGULARITY, tf.float32), (TOP_PACK, tf.string), (FREQ_TOP_PACK, tf.float32)]), tf.int32)>

In [10]:
def create_feature_columns():
  k_region = tf.keras.Input(name="REGION" , shape=(1,) , dtype= tf.string)
  k_tenure = tf.keras.Input(name="TENURE" , shape=(1,) , dtype= tf.string)
  k_montant = tf.keras.Input(name="MONTANT" , shape=(1,) , dtype= tf.float64)
  k_freq_rech = tf.keras.Input(name="FREQUENCE_RECH" , shape=(1,) , dtype= tf.float64)
  k_revenue = tf.keras.Input(name="REVENUE" , shape=(1,) , dtype= tf.float64)
  k_arpu_seg = tf.keras.Input(name="ARPU_SEGMENT" , shape=(1,) , dtype= tf.float64)
  k_freq = tf.keras.Input(name="FREQUENCE" , shape=(1,) , dtype= tf.float64)
  k_data_vol = tf.keras.Input(name="DATA_VOLUME" , shape=(1,) , dtype= tf.float64)
  k_on_net = tf.keras.Input(name="ON_NET" , shape=(1,) , dtype= tf.float64)
  k_orange = tf.keras.Input(name="ORANGE" , shape=(1,) , dtype= tf.float64)
  k_tigo = tf.keras.Input(name="TIGO" , shape=(1,) , dtype= tf.float64)
  k_zone1 = tf.keras.Input(name="ZONE1" , shape=(1,) , dtype= tf.float64)
  k_zone2 = tf.keras.Input(name="ZONE2" , shape=(1,) , dtype= tf.float64)
  k_mrg = tf.keras.Input(name="MRG" , shape=(1,) , dtype= tf.string)
  k_regularity = tf.keras.Input(name="REGULARITY" , shape=(1,) , dtype= tf.float64)
  k_top_pack = tf.keras.Input(name="TOP_PACK" , shape=(1,) , dtype= tf.string)
  k_freq_top_pack = tf.keras.Input(name="FREQ_TOP_PACK" , shape=(1,) , dtype= tf.float64)

  keras_input_dict = {'REGION':k_region, 'TENURE':k_tenure, 'MONTANT':k_montant, 
                      'FREQUENCE_RECH':k_freq_rech, 'REVENUE':k_revenue ,
                      'ARPU_SEGMENT':k_arpu_seg, 'FREQUENCE':k_freq, 
                      'DATA_VOLUME':k_data_vol, 'ON_NET':k_on_net, 'ORANGE':k_orange, 
                      'TIGO':k_tigo, 'ZONE1':k_zone1, 'ZONE2':k_zone2, 'MRG':k_mrg, 
                      'REGULARITY':k_regularity, 'TOP_PACK':k_top_pack, 
                      'FREQ_TOP_PACK':k_freq_top_pack
                      }

  return ({'K':keras_input_dict})

create_feature_columns()

{'K': {'ARPU_SEGMENT': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'ARPU_SEGMENT')>,
  'DATA_VOLUME': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'DATA_VOLUME')>,
  'FREQUENCE': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'FREQUENCE')>,
  'FREQUENCE_RECH': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'FREQUENCE_RECH')>,
  'FREQ_TOP_PACK': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'FREQ_TOP_PACK')>,
  'MONTANT': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'MONTANT')>,
  'MRG': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'MRG')>,
  'ON_NET': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'ON_NET')>,
  'ORANGE': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'ORANGE')>,
  'REGION': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'REGION')>,
  'REGULARITY': <KerasTensor: shape=(None, 1) dtype=float64 (created by layer 'REGU

In [11]:

def create_model(feature_columns):
  METRICS = [keras.metrics.MeanAbsoluteError(name='mae')]
  input_features = []
  counter = 0
  for inp in feature_columns['K'].keys():
    input_features.append(feature_columns['K'][inp])


  # Categorical features preprocessing
  cat_tenure = tf.keras.layers.StringLookup(vocabulary=unique_tenures, mask_token=None)(feature_columns['K']['TENURE'])
  cat_tenure = tf.keras.layers.CategoryEncoding(num_tokens=len(unique_tenures)+3, output_mode='one_hot')(cat_tenure)
  
  cat_region = tf.keras.layers.StringLookup(vocabulary=unique_regions, mask_token=None)(feature_columns['K']['REGION'])
  cat_region = tf.keras.layers.CategoryEncoding(num_tokens=len(unique_regions)+3, output_mode='one_hot')(cat_region)
  #cat_top_pack = tf.keras.layers.experimental.preprocessing.CategoricalEncoding(max_tokens=200)(feature_columns['K']['TOP_PACK'])
  
  # Numerical values
  all_numeric_features = tf.keras.layers.concatenate([
                        feature_columns['K']['MONTANT'], feature_columns['K']['FREQUENCE_RECH'],
                        feature_columns['K']['REVENUE'], feature_columns['K']['ARPU_SEGMENT'],
                        feature_columns['K']['FREQUENCE'], feature_columns['K']['DATA_VOLUME'],
                        feature_columns['K']['ON_NET'], feature_columns['K']['ORANGE'],
                        feature_columns['K']['ZONE1'], feature_columns['K']['ZONE2'],
                        feature_columns['K']['REGULARITY'], feature_columns['K']['FREQ_TOP_PACK'],
                        ])
  
  all_categorical_columns = tf.keras.layers.concatenate([cat_tenure, cat_region])

  x_numeric = tf.keras.layers.Dense(32, activation='sigmoid', input_shape=(100,), kernel_initializer="he_uniform")(all_numeric_features)
  x_numeric = tf.keras.layers.BatchNormalization()(x_numeric)

  x_categorical = tf.keras.layers.Dense(32, activation='sigmoid', kernel_initializer="he_uniform")(all_categorical_columns)
  
  x = tf.keras.layers.concatenate([x_numeric, x_categorical])
  
  # Creating the model
  '''
  x = tf.keras.layers.Reshape((64,1))(x)
 
  x = tf.keras.layers.LSTM(20, return_sequences=True, dropout=0.2)(x)
  x = tf.keras.layers.LSTM(20, return_sequences=True, dropout=0.2)(x)
  x = tf.keras.layers.LSTM(20, dropout=0.2)(x)
  
  x = tf.keras.layers.Flatten()(x)
  '''

  x = tf.keras.layers.Dense(32, activation='sigmoid', kernel_initializer="he_uniform",
                                  activity_regularizer=tf.keras.regularizers.l2(0.00001))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  
  out = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(input_features, out)

  opt = tf.keras.optimizers.Adam(lr = 0.002)

  #Compile the model
  model.compile(loss='binary_crossentropy',optimizer=opt, metrics = METRICS )

  print(model.summary())
  print(model.output_shape)
  return model



In [12]:
def train_and_evaluate(model, training_dataset , validation_dataset, epochs = 10):
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001, verbose = 1)
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

  #Train and Evaluate
  out = model.fit(training_dataset, 
                  validation_data = validation_dataset,
                  epochs=epochs,
                  # validation_steps = 3,   ###Keep this none for running evaluation on full EVAL data every epoch
                  steps_per_epoch = 100,   ###Has to be passed - Cant help it :) [ Number of batches per epoch ]
                  callbacks=[reduce_lr, #modelsave_callback, #tensorboard_callback, 
                             keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=True)]
                  )


In [13]:
#Create dataset input functions
train_dataset = create_input_fn(filename = "/content/drive/MyDrive/Expresso Churn/tfTrain.csv",
                    mode = tf.estimator.ModeKeys.TRAIN,
                    # vnum_epochs = epochs,
                    batch_size = 128)()

validation_dataset = create_input_fn(filename = "/content/drive/MyDrive/Expresso Churn/tfEval.csv",
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 512)()

m_ = create_model(feature_columns = create_feature_columns())
#tf.keras.utils.plot_model(m_, show_shapes=True, rankdir="LR")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 TENURE (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 REGION (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 MONTANT (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 FREQUENCE_RECH (InputLayer)    [(None, 1)]          0           []                               
                                                                                              

In [14]:
train_and_evaluate(m_, train_dataset, validation_dataset, 30)

Epoch 1/30
100/100 [==============================] - 18s 155ms/step - loss: 0.4195 - mae: 0.2838 - val_loss: 0.4214 - val_mae: 0.2998 - lr: 0.0020
Epoch 2/30
100/100 [==============================] - 15s 153ms/step - loss: 0.2880 - mae: 0.1924 - val_loss: 0.3671 - val_mae: 0.2249 - lr: 0.0020
Epoch 3/30
100/100 [==============================] - 24s 238ms/step - loss: 0.2805 - mae: 0.1819 - val_loss: 0.3102 - val_mae: 0.2110 - lr: 0.0020
Epoch 4/30
100/100 [==============================] - 22s 221ms/step - loss: 0.2753 - mae: 0.1737 - val_loss: 0.2815 - val_mae: 0.1943 - lr: 0.0020
Epoch 5/30
100/100 [==============================] - 13s 134ms/step - loss: 0.2753 - mae: 0.1736 - val_loss: 0.3035 - val_mae: 0.1902 - lr: 0.0020
Epoch 6/30
100/100 [==============================] - 13s 132ms/step - loss: 0.2755 - mae: 0.1744 - val_loss: 0.2791 - val_mae: 0.1937 - lr: 0.0020
Epoch 7/30
100/100 [==============================] - 13s 133ms/step - loss: 0.2772 - mae: 0.1721 - val_loss: 0.

In [15]:
#Trying out an example
data = tf.data.Dataset.from_tensor_slices({
    'REGION': [['THIES']],
    'TENURE': [['K > 24 month']],
    'MONTANT':[[1000]],
    'FREQUENCE_RECH':[[2]],
    'REVENUE':[[990]],
    'ARPU_SEGMENT':[[330]],
    'FREQUENCE':[[3]],
    'DATA_VOLUME':[[2268]],
    'ON_NET':[[7]],
    'ORANGE':[[10]],
    'TIGO':[[4]],
    'ZONE1':[[0]],
    'ZONE2':[[0]],
    'MRG':[['NO']],
    'REGULARITY':[[8]],
    'TOP_PACK':[['Data:490F=1GB,7d']],
    'FREQ_TOP_PACK':[[2]]
})
m_.predict(data)


array([[0.0103]], dtype=float32)

In [16]:
@tf.function
def serving(region,tenure,montant,frequence_rech, revenue, 
            arpu_segment, frequence, data_volume, on_net, orange, 
            tigo, zone1, zone2, mrg, regularity, top_pack, freq_top_pack):
  payload = {
      'REGION': region,
      'TENURE': tenure,
      'MONTANT':montant,
      'FREQUENCE_RECH':frequence_rech,
      'REVENUE':revenue,
      'ARPU_SEGMENT':arpu_segment,
      'FREQUENCE':frequence,
      'DATA_VOLUME':data_volume,
      'ON_NET':on_net,
      'ORANGE':orange,
      'TIGO':tigo,
      'ZONE1':zone1,
      'ZONE2':zone2,
      'MRG':mrg,
      'REGULARITY':regularity,
      'TOP_PACK':top_pack,
      'FREQ_TOP_PACK':freq_top_pack,      
      }
    
  predictions = m_(payload)
  return predictions

In [17]:
serving = serving.get_concrete_function(region=tf.TensorSpec([None,], dtype= tf.string, name='region'),
                                        tenure=tf.TensorSpec([None,], dtype= tf.string, name='tenure'),
                                        montant=tf.TensorSpec([None,], dtype= tf.float64, name='montant'),
                                        frequence_rech=tf.TensorSpec([None,], dtype= tf.float64, name='frequence_rech'), 
                                        revenue=tf.TensorSpec([None,], dtype= tf.float64, name='revenue'), 
                                        arpu_segment=tf.TensorSpec([None,], dtype= tf.float64, name='arpu_segment'), 
                                        frequence=tf.TensorSpec([None,], dtype= tf.float64, name='frequence'), 
                                        data_volume=tf.TensorSpec([None,], dtype= tf.float64, name='data_volume'), 
                                        on_net=tf.TensorSpec([None,], dtype= tf.float64, name='on_net'), 
                                        orange=tf.TensorSpec([None,], dtype= tf.float64, name='orange'), 
                                        tigo=tf.TensorSpec([None,], dtype= tf.float64, name='tigo'), 
                                        zone1=tf.TensorSpec([None,], dtype= tf.float64, name='zone1'), 
                                        zone2=tf.TensorSpec([None,], dtype= tf.float64, name='zone2'), 
                                        mrg=tf.TensorSpec([None,], dtype= tf.string, name='mrg'), 
                                        regularity=tf.TensorSpec([None,], dtype= tf.float64, name='regularity'), 
                                        top_pack=tf.TensorSpec([None,], dtype= tf.string, name='top_pack'), 
                                        freq_top_pack=tf.TensorSpec([None,], dtype= tf.float64, name='freq_top_pack')
                                        )

version = "1"  #{'serving_default': call_output}
tf.saved_model.save(
    m_,
    "/content/drive/MyDrive/Expresso Churn/expresso_churn_model/" + version,
    signatures=serving
) 


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/assets


In [18]:
!tar -cvf /content/model.tar '/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/'

tar: Removing leading `/' from member names
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/variables/
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/variables/variables.index
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/variables/variables.data-00000-of-00001
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/assets/
/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1/saved_model.pb


In [19]:
##Check signature
!saved_model_cli show --dir "/content/drive/MyDrive/Expresso Churn/expresso_churn_model/1" --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['arpu_segment'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: serving_default_arpu_segment:0
    inputs['data_volume'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: serving_default_data_volume:0
    inputs['freq_top_pack'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: serving_default_freq_top_pack:0
    inputs['frequence'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1)
        name: serving_defaul

In [20]:
#LOCAL: Predict using Keras prediction function
saved_mod = tf.saved_model.load("/content/drive/MyDrive/Expresso Churn/expresso_churn_model/" + version)

#Get prediction function from serving
f = saved_mod.signatures['serving_default']

#Run prediction function from serving
# Casting all numeric to double since it was what we used to create the model
# results ini an error otherwise
f(region= tf.convert_to_tensor(['THIES']),
    tenure= tf.convert_to_tensor(['K > 24 month']),
    montant= tf.cast(tf.convert_to_tensor([1000]), tf.float64),
    frequence_rech= tf.cast(tf.convert_to_tensor([2]), tf.float64),
    revenue = tf.cast(tf.convert_to_tensor([990]), tf.float64),
    arpu_segment = tf.cast(tf.convert_to_tensor([330]), tf.float64),
    frequence = tf.cast(tf.convert_to_tensor([3]), tf.float64),
    data_volume = tf.cast(tf.convert_to_tensor([2268]), tf.float64),
    on_net = tf.cast(tf.convert_to_tensor([7]), tf.float64),
    orange = tf.cast(tf.convert_to_tensor([10]), tf.float64),
    tigo = tf.cast(tf.convert_to_tensor([4]), tf.float64),
    zone1 = tf.cast(tf.convert_to_tensor([0]), tf.float64),
    zone2 = tf.cast(tf.convert_to_tensor([0]), tf.float64),
    mrg = tf.convert_to_tensor(['NO']),
    regularity = tf.cast(tf.convert_to_tensor([8]), tf.float64),
    top_pack = tf.convert_to_tensor(['Data:490F=1GB,7d']),
    freq_top_pack = tf.cast(tf.convert_to_tensor([2]), tf.float64)
    )

{'output_0': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.0103]], dtype=float32)>}

In [21]:
###Install TF Model server

# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!apt-get remove tensorflow-model-server
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  tensorflow-model-server
0 upgraded, 0 newly installed, 1 to remove and 100 not upgraded.
After this operation, 0 B of additional disk space will be used.
(Reading database ... 155223 files and directories currently installed.)
Removing tensorflow-model-server (2.7.0) ...
deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  89181      0 --:--:-- --:--:-- --:--:-- 91968
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x8

In [22]:
###Start Tensorflow server
# %%bash --bg 
# export TF_CPP_MIN_VLOG_LEVEL=0

%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8505 \
  --model_name=model \
  --model_base_path="/content/drive/MyDrive/Expresso Churn/expresso_churn_model" >server.log 2>&1

Starting job # 0 in a separate thread.


In [23]:
import json
import requests

#Create payload
data_py = {"instances":[{'region': 'THIES',
                        'tenure': 'K > 24 month',
                        'montant':10,
                        'frequence_rech':20,
                        'revenue':9,
                        'arpu_segment':300,
                        'frequence':30,
                        'data_volume':22,
                        'on_net':20,
                        'orange':10,
                        'tigo':40,
                        'zone1':0,
                        'zone2':0,
                        'mrg':'NO',
                        'regularity':8,
                        'top_pack':'Data:490F=1GB,7d',
                        'freq_top_pack':20
                         }]}

data = json.dumps(data_py)
#print("payload: ", data)

#Run request on TMS
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8505/v1/models/model:predict', data=data, headers=headers)
json_response.text

'{\n    "predictions": [[0.339752972]\n    ]\n}'